In [60]:
import os
import pandas as pd
import urllib
import sagemaker, boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer
from sklearn.model_selection import train_test_split

# Create S3 bucket

In [16]:
region=boto3.session.Session().region_name
region

'eu-north-1'

In [18]:
s3 = boto3.resource('s3')
bucket_name = "marzieh-project"  

# Create an S3 bucket with the specified region
s3.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={'LocationConstraint': region})

s3.Bucket(name='marzieh-project')

In [19]:
prefix = 'xgboost'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://marzieh-project/xgboost/output


# Downloading dataset and store in S3

In [37]:
urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv",
                            "bank_clean.csv")

model_data = pd.read_csv('./bank_clean.csv',index_col=0)
train,test=train_test_split(model_data,test_size=0.3,random_state=1729)

pd.concat([train['y_yes'], train.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)

pd.concat([test['y_yes'], test.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('test.csv', index=False, header=False)


boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')


s3_input_train = TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')
s3_input_test = TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

# Building models Xgboot

In [47]:
container = get_image_uri(boto3.Session().region_name,'xgboost')   

hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }   

estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          output_path=output_path,
                                          role=sagemaker.get_execution_role(),
                                          train_instance_count=1, 
                                          train_instance_type='ml.m5.2xlarge', 
                                          train_volume_size=5, # 5 GB                                           
                                          train_use_spot_instances=True,
                                          train_max_run=300,
                                          train_max_wait=600)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


# Traning and Depoly the model

In [48]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: xgboost-2023-12-08-10-23-29-647


2023-12-08 10:23:29 Starting - Starting the training job......
2023-12-08 10:24:02 Starting - Preparing the instances for training......
2023-12-08 10:25:25 Downloading - Downloading input data...
2023-12-08 10:26:01 Training - Downloading the training image...
2023-12-08 10:26:11 Training - Training image download completed. Training in progress..Arguments: train
[2023-12-08:10:26:29:INFO] Running standalone xgboost training.
[2023-12-08:10:26:29:INFO] File size need to be processed in the node: 4.83mb. Available memory size in the node: 23618.02mb
[2023-12-08:10:26:29:INFO] Determined delimiter of CSV input is ','
[10:26:29] S3DistributionType set as FullyReplicated
[10:26:30] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-12-08:10:26:30:INFO] Determined delimiter of CSV input is ','
[10:26:30] S3DistributionType set as FullyReplicated
[10:26:30] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/v

In [54]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [61]:
test_array = test.drop(['y_no', 'y_yes'], axis=1).values 
xgb_predictor.content_type = 'text/csv' 
xgb_predictor.serializer = CSVSerializer

predictions = xgb_predictor.predict(test_array).decode('utf-8') 
predictions_array = np.fromstring(predictions[1:], sep=',') 
print(predictions_array)

NameError: name 'xgb_predictor' is not defined

In [59]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()